## ETL (proceso de transformacion)

In [12]:
#importamos los modulos y librerias

import pandas as pd
import numpy as np
from ast import literal_eval
import ast

In [4]:
#carga de los datasets

movies = pd.read_csv('/content/drive/MyDrive/Kaarla,/Dataset/movies_dataset.csv')
creditos = pd.read_csv('/content/drive/MyDrive/Kaarla,/Dataset/credits.csv')

<ipython-input-4-b7b06037435b>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('/content/drive/MyDrive/Kaarla,/Dataset/movies_dataset.csv')


In [5]:
pd.options.display.max_columns=0 #vista previa
movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [6]:

creditos.head(2) #vista previa

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [7]:
# generamos copia de los archivos

creditos1=creditos.copy()
movie=movies.copy()

In [9]:
# verificamos los duplicados en creditos1

creditos1.duplicated().value_counts()

False    45439
True        37
dtype: int64

In [10]:
#eliminacion de duplicados
creditos1=creditos1.drop_duplicates()

In [11]:
creditos1=creditos1.reset_index(drop=True)  #reindexamos el dataframe

In [13]:
# conversion de str a lista de diccionarios por fila : cast and crew

creditos1['cast']= creditos1.cast.apply(literal_eval)
creditos1['crew']=creditos1.crew.apply(literal_eval)

In [14]:
creditos1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45439 entries, 0 to 45438
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45439 non-null  object
 1   crew    45439 non-null  object
 2   id      45439 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
# adult =adulto
# belongs_to_collection	= pertenece a la coleccion
# budget =presupuesto
# genres = generos
# homepage =pagina principal
# id =identificacion
# imbd id = identificacion imbd
# original_language = idioma original
# original_title =titulo original
# overwiev =descripcion original
# popularity =popularidad
# poster_path = link del cartel o direccion
# production_companies = compañias de produccion
# production_countries =paises de produccion
# fecha de lanzamiento =

In [15]:
# veificamos si hay duplicados en movie

movie.duplicated().value_counts()

False    45453
True        13
dtype: int64

In [ ]:
movie[movie.duplicated(keep=False)].head(3)# se muestra las filas o instancias duplicadas

In [18]:
movie=movie.drop_duplicates() #eliminamos los duplicados : al considerarlos en nuestro analisis no posibilita definir algunos parametros

In [20]:
movie=movie.reset_index(drop=True) # hacemos la reindexacion

In [21]:
movie.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [22]:
movie.isna().sum()

adult                        0
belongs_to_collection    40959
budget                       0
genres                       0
homepage                 37673
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

##transformacion de los campos anidados en el dataframe movie

In [23]:
#columnas anidadas de diccionarios : belong_collection

In [24]:

def conversion_columnas_dict(columna):
    listObj=[]
    for i in range(len(columna)):
        if type(columna[i])== str:
             listObj.append(ast.literal_eval(columna[i]))
        else:
           listObj.append(ast.literal_eval("{'id': 0, 'name': 'no_definido', 'poster_path': 'no_definido', 'backdrop_path': 'no_definido'}"))
    serie = pd.Series(listObj)
    return serie

In [25]:
# iterar solo una vez
movie['belongs_to_collection']=conversion_columnas_dict(movie['belongs_to_collection']) #cada elemento del a columna belong es un diccionario


In [33]:
# columnas anidadas lista de diccionarios : gnere ,production_companies,production_countries,spoken_languages

import ast
import numpy as np

def conversion_columnas_list(columna):
    c=columna
    listObj=[]
    for i in range(len(columna)):
        if type(columna[i])== str:
           if columna[i]=='[]':
              listObj.append("[{'name': 'sin_dat', 'id': 0}]")
           elif columna[i]=='False':
                listObj.append("[{'name': 'sin_dat', 'id': 0}]")
           elif columna[i][1]=='.':
                listObj.append("[{'name': 'sin_dat', 'id': 0}]")
           else:
             listObj.append(columna[i])
        else:
           listObj.append("[{'name': 'sin_dat', 'id': 0}]")
    serie = pd.Series(listObj)
    return serie

In [34]:
# datos vacios completados con la estrctura "[{'name': 'sin_dat', 'id': 0}]"
movie['genres']=conversion_columnas_list(movie['genres'])
movie['production_companies']=conversion_columnas_list(movie['production_companies'])
movie['production_countries']=conversion_columnas_list(movie['production_countries'])
movie['spoken_languages']=conversion_columnas_list(movie['spoken_languages'])

In [35]:
# conversion de srt a listta de diiccionarios por fila
movie['genres']= movie.genres.apply(literal_eval)
movie['production_companies']= movie.production_companies.apply(literal_eval)
movie['production_countries']= movie.production_countries.apply(literal_eval)
movie['spoken_languages']= movie.spoken_languages.apply(literal_eval)

In [36]:
movie = movie.drop(labels=[19725,29491,35575],axis=0)  # se eliminan las filas  ['1', '12', '22' ]  ,  datos incompletos e imprecisos

In [37]:
## conversion de tipo str a float  :   budget ,id,popularity ,revenue,runtime,vote_average,vote_count
movie['budget'] = movie['budget'].astype(float)
movie['id'] = movie['id'].astype(int)
movie['popularity'] = movie['popularity'].astype(float)
movie['revenue'] = movie['revenue'].astype(float)
movie['runtime'] = movie['runtime'].astype(float)
movie['vote_average'] = movie['vote_average'].astype(float)
movie['vote_count'] = movie['vote_count'].astype(float)

In [38]:
movie=movie.reset_index(drop=True) #reindexamos reordenamos los indices

## valores nulos de los campos revenue y budget rellenados con 0

In [40]:

# reemplazamos los nan a cero en la columna revenue (ganacia)y budget(persopue) para un posterior calculo

movie['revenue'] = movie['revenue'].fillna(0)
movie['budget'] = movie['budget'].fillna(0)

In [41]:
# eliminamos los nulos de la columna relaese_date al realizar el analisi estadistico de las variables numericas estas no presentan una variacion significativa en sus parametros
# por tanto no influye en el analisis

movie=movie[movie.release_date.notnull()]

In [42]:
movie=movie.reset_index(drop=True) #reindexamos

## conversion de la columna release_date a formato fecha

In [43]:
# convertimos la columna release_date a fecha

movie['release_date'] = pd.to_datetime(movie['release_date'], format='%Y-%m-%d')

In [45]:
# definimos una columna year a partir de la extraccion de año de release_date

movie['release_year'] = movie['release_date'].dt.year

## crear la columna con el retorno de inversion : revenue/budget = return

In [46]:
# definimos la columna return
def divisionentrecolumnas(columna1,columna2):
  h=[]
  for i in range(len(columna1)):
      if columna1[i]!= 0 :
        if  columna2[i]!= 0:
            h.append(columna1[i]/columna2[i])
        else:
           h.append(0)
      else :
            h.append(0)
  serie = pd.Series(h)
  return serie

In [47]:

#generamos la variable return

movie['return'] =divisionentrecolumnas(movie['revenue'], movie['budget'])

# eliminamos las columnas : video, imdb_id, adult,
# original_title , poster_path,homepage

In [49]:
# eliminamos las columnas que no sean relevantes para nuestro estudio ,eliminamos las columnas que no seran utilizdas

movie=movie.drop(columns=['video', 'imdb_id','adult','original_title','poster_path','homepage'])



 ## El merge de los dataframes transformados

In [50]:
df=pd.merge(movie,creditos1, on='id')


In [51]:
df.head(5)

,belong_collection.id,belong_collection.name,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,10194.0,Toy Story Collection,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,0.0,no_definido,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,119050.0,Grumpy Old Men Collection,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,0.0,no_definido,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,96871.0,Father of the Bride Collection,0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [52]:
df.shape

(45375, 23)

In [57]:
df.isna().sum()

belong_collection.id          0
belong_collection.name        0
budget                        0
genres                        0
id                            0
original_language            11
overview                    941
popularity                    0
production_companies          0
production_countries          0
release_date                  0
revenue                       0
runtime                     246
spoken_languages              0
status                       80
tagline                   24977
title                         0
vote_average                  0
vote_count                    0
release_year                  0
return                        0
cast                          0
crew                          0
dtype: int64

In [62]:
#guardamos el data frame en un csv con nombre movies_consulta

df.to_csv('movies_consulta.csv', encoding='utf-8', index=False)